For demonstration purposes, let us use a larger dataset to test the **Dense Passage Retrieval (DPR)** pipeline, we can use pre-built datasets that are commonly used for Question Answering (QA) tasks. One of the most popular datasets for this purpose is **Natural Questions**. These datasets are quite large and provide both questions and passages to test retrieval and reading models.

**Natural Questions (NQ) Dataset:**

The **Natural Questions (NQ)** dataset contains real user queries along with corresponding passages retrieved from Wikipedia. It's a great dataset for testing retrieval models.

To demonstrate the DPR pipeline with a larger dataset, we'll use a portion of the **Natural Questions Open (NQ)** dataset, which is publicly available via Hugging Face's `datasets` library.

In [1]:
# Step 1: Install Haystack
!pip install farm-haystack[inference] --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.9/763.9 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━

In [2]:
# Step 1: Install Haystack
!pip install farm-haystack[faiss] datasets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework

In [3]:
# Step 2: Import Required Libraries
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import DensePassageRetriever, FARMReader
from haystack.pipelines import ExtractiveQAPipeline
from haystack.utils import print_answers
from datasets import load_dataset


In [4]:
# Step 3: Set Up the FAISS Document Store
document_store = FAISSDocumentStore(embedding_dim=768)  # 768 is the embedding dimension used by DPR
# Please do not run it 2nd time. It will raise an exception.
# In case you need to rerun First delete the current runtime then run all.

In [5]:
# Step 4: Load NQ Dataset

# Load the 'train' split of Natural Questions Open dataset (subset)
nq_dataset = load_dataset("natural_questions", split="train[:50]")

# Convert the dataset into a format compatible with Haystack
documents = [{"content": item['context']} for item in nq_dataset]

print(f"Loaded {len(documents)} documents.")


# Write the documents to the document store
document_store.write_documents(documents)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/13.7k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/287 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/287 [00:00<?, ?it/s]

train-00000-of-00287.parquet:   0%|          | 0.00/192M [00:00<?, ?B/s]

...

validation-00004-of-00007.parquet:   0%|          | 0.00/196M [00:00<?, ?B/s]

validation-00005-of-00007.parquet:   0%|          | 0.00/190M [00:00<?, ?B/s]

validation-00006-of-00007.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/307373 [00:00<?, ? examples/s]

DatasetGenerationError: An error occurred while generating the dataset

In [ ]:
# Step 5: Initialize the Dense Passage Retriever (DPR)

"""
The Dense Passage Retriever (DPR) retrieves documents using dense embeddings.
DPR requires two models:

Query Embedding Model:   Embeds the query into a vector space.
Passage Embedding Model: Embeds the documents into the same vector space
                         for similarity comparison.
"""
# Initialize Dense Passage Retriever for dense vector-based retrieval
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    use_gpu=False  # Set to True if you want to use a GPU
)

# Update the document store with embeddings for the documents
document_store.update_embeddings(retriever)


In [ ]:
# Step 6: Initialize the Reader
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=False)

In [ ]:
# Step 7: Build the Pipeline
pipeline = ExtractiveQAPipeline(reader=reader, retriever=retriever)


In [ ]:
# Step 8: Ask a Question and Get an Answer

query = "Where is the capital of France?"

# Run the pipeline and get answers
prediction = pipeline.run(query=query, params={"Retriever": {"top_k": 1}, "Reader": {"top_k": 1}})

# Print the answers
print_answers(prediction, details="minimum")


In [ ]:
# Step 9: Test with Another Query

query_2 = "Who developed the theory of relativity?"

# Run the pipeline and get answers
prediction_2 = pipeline.run(query=query_2, params={"Retriever": {"top_k": 1}, "Reader": {"top_k": 1}})

# Print the answers
print_answers(prediction_2, details="minimum")
